# Overview

In [1]:
library(reticulate)

In [2]:
sklearn <- import("sklearn")
datasets <- import("sktime.datasets")
np <- import("numpy")

# Prepare data

### Load dataset

In [25]:
arrow_head <- datasets$load_basic_motions(split="train", return_X_y=TRUE)

In [26]:
x_train <- arrow_head[[1]]
y_train <- arrow_head[[2]]

### Truncate some of the time series in order to complicate the problem

- 33.(3)% of sequences left at 50% length
- 33.(3)% of sequences truncated to 40% length
- 33.(3)% of sequences truncated to 30% length

#### In order to ensure that all classes are treated equally stratyfikacja

In [21]:
time_series_count = length(x_train[[1]])            # number of time series in the dataset
time_series_length = length(x_train[1,][[1]][[1]])  # number of observations in each time serie
time_series_dims = length(x_train[1,])              # number of dimensions of each time serie

first_index = 1                                     # index of first element of time series with 100% length
second_index = ceiling(time_series_count / 3)       # index of first element of time series with 75% length
third_index = ceiling(2 * time_series_count / 3)    # index of first element of time series with 50% length

mask = 0 : (time_series_length - 1)
for (i in first_index : (second_index - 1))
    for (j in 1 : time_series_dims)
        x_train[i,][[j]][[1]] = x_train[i,][[j]][[1]][mask]

mask = sort(sample(0 : (time_series_length - 1), ceiling(85 * time_series_length / 100))) 
for (i in second_index : (third_index - 1))
    for (j in 1 : time_series_dims)
        x_train[i,][[j]][[1]] = x_train[i,][[j]][[1]][mask]

    
mask = sort(sample(0 : (time_series_length - 1), ceiling(75 * time_series_length / 100)))
for (i in third_index : time_series_count)
    for (j in 1 : time_series_dims) 
        x_train[i,][[j]][[1]] = x_train[i,][[j]][[1]][mask]

# ROCKET

### Standarize time series length by repeating observations

In [22]:
fix_length <- function(x, expected_length) {
    current_length = length(x)
    
    if (current_length != expected_length) {
        x_args = strtoi(stringr::str_trim(names(x)), 10)
        y_args = as.vector(x)
        return(approx(x_args, y_args, 0:(expected_length - 1))$y)
    }
    
    return(x)
}

In [23]:
max_time_series_length = time_series_length
time_series_count = length(x_train[[1]])
time_series_dims = length(x_train[1,])

for (i in 1 : time_series_count)
    for (j in 1 : time_series_dims)
        x_train[i,][[j]][[1]] = fix_length(x_train[i,][[j]][[1]], max_time_series_length)

In [24]:
utils <- import("sktime.utils.validation.panel")
x_train = utils$check_X(x_train, coerce_to_numpy=TRUE)

### Generate kernels

In [39]:
arrow_head <- datasets$load_basic_motions(split="train", return_X_y=TRUE)
x_train <- arrow_head[[1]]
y_train <- arrow_head[[2]]
utils <- import("sktime.utils.validation.panel")
x_train = utils$check_X(x_train, coerce_to_numpy=TRUE)




In [40]:
for(i in 1 : dim(x_train)[1])
    for (o in 1 : dim(x_train)[2]) {
        x_train[i,o,] = (x_train[i,o,] - mean(x_train[i,o,])) / sd(x_train[i,o,]) + 1e-8 
    }

In [41]:
set.seed(5)
num_kernels = 10000 # default value

num_columns = dim(x_train)[2]
num_timepoints = dim(x_train)[3]

lengths = array(as.integer(sample(c(7, 9, 11), num_kernels, replace = TRUE)))

limit = pmin(num_columns, lengths)

num_channel_indices = as.integer(2 ** np$random$uniform(0, log2(limit + 1)))

channel_indices = as.integer(rep(0, sum(num_channel_indices)))

weights = as.double(rep(0, sum(lengths * num_channel_indices)))

biases = array(as.double(rep(0, num_kernels)))
dilations = array(as.integer(rep(0, num_kernels)))
paddings = array(as.integer(rep(0, num_kernels)))

a1 = 1  # for weights
a2 = 1  # for channel_indices

for (i in 1 : num_kernels) {
    temp_length = lengths[i]
    temp_num_channel_indices = num_channel_indices[i]

    temp_weights = as.double(np$random$normal(0, 1, temp_num_channel_indices * temp_length))

    b1 = a1 + (temp_num_channel_indices * temp_length) - 1 
    b2 = a2 + temp_num_channel_indices - 1

    a3 = 1 # for weights (per channel)
    for (j in 1 : temp_num_channel_indices) {
        b3 = a3 + temp_length - 1
        temp_weights[a3 : b3] = temp_weights[a3 : b3] - mean(temp_weights[a3 : b3])
        a3 = b3 + 1
    }
        

    weights[a1 : b1] = temp_weights

    channel_indices[a2 : b2] = sample(0 : (num_columns - 1), temp_num_channel_indices)

    biases[i] = np$random$uniform(-1, 1)

    dilation = 2 ** np$random$uniform(0, log2((num_timepoints - 1) / (temp_length - 1)))
    dilation = as.integer(dilation)
    dilations[i] = dilation

    if (sample(0 : 1, 1) == 1)
        paddings[i] = as.integer(((temp_length - 1) * dilation) / 2)
    else paddings[i] = 0

    a1 = b1 + 1
    a2 = b2 + 1
} 

channel_indices = channel_indices + 1

### Transform x_train with created kernels

#### Normalize x_train

#### Transform x_train

In [42]:
apply_kernel_multivariate <- function(X, 
                                      weights, 
                                      length, 
                                      bias, 
                                      dilation, 
                                      padding, 
                                      num_channel_indices, 
                                      channel_indices) {
    
    num_columns = dim(X)[1]
    num_timepoints = dim(X)[2]

    output_length = (num_timepoints + (2 * padding)) - ((length - 1) * dilation)

    temp_ppv = 0
    temp_max = -Inf

    end = (num_timepoints + padding) - ((length - 1) * dilation) - 1
    
    for (i in (-padding + 1) : end) 
    {
        temp_sum = bias

        index = i

        for (j in 1 : length) {
            if (index > 0 && index <= num_timepoints) {
                for (k in 1 : num_channel_indices) {
                    temp_sum = temp_sum + weights[k, j] * X[channel_indices[k], index]
                }
                    
            }
            index = index + dilation
        }
            
        if (temp_sum > temp_max)
            temp_max = temp_sum

        if (temp_sum > 0)
            temp_ppv = temp_ppv + 1 
    }
    return(c(as.double(temp_ppv / output_length), as.double(temp_max)))
}


apply_kernel_univariate <- function(X, weights, length, bias, dilation, padding) {
    num_timepoints = length(X)

    output_length = (num_timepoints + (2 * padding)) - ((length - 1) * dilation)

    temp_ppv = 0
    temp_max = -Inf

    end = (num_timepoints + padding) - ((length - 1) * dilation)

    for (i in (-padding + 1) : end) 
    {
        temp_sum = bias

        index = i

        for (j in 1 : length) {
            if (index > 0 && index <= num_timepoints)
                temp_sum = temp_sum + weights[j] * X[index]

            index = index + dilation

        }
            
        if (temp_sum > temp_max)
            temp_max = temp_sum

        if (temp_sum > 0)
            temp_ppv = temp_ppv + 1
    }
    return(c(as.double(temp_ppv / output_length), as.double(temp_max)))
}


In [43]:
num_instances = dim(x_train)[1]
num_columns = dim(x_train)[2]

num_kernels = length(lengths)

X = matrix(0, num_instances, num_kernels * 2)  # 2 features per kernel

for (i in 1 : num_instances) {
    a1 = 1  # for weights
    a2 = 1  # for channel_indices
    a3 = 1  # for features

    for (j in 1 : num_kernels)
    {
        b1 = a1 + num_channel_indices[j] * lengths[j] - 1
        b2 = a2 + num_channel_indices[j] - 1
        b3 = a3 + 2 - 1
        
        if (num_channel_indices[j] == 1)
        {
            
            temp_result = apply_kernel_univariate(
                x_train[i, channel_indices[a2],],
                weights[a1:b1],
                lengths[j],
                biases[j],
                dilations[j],
                paddings[j]
            ) 
            X[i, a3] = temp_result[1]
            X[i, a3 + 1] = temp_result[2]
        }
        else
        {
            temp_weights = matrix(weights[a1 : b1], num_channel_indices[j], lengths[j])

            temp_result = apply_kernel_multivariate(
                x_train[i,,],
                temp_weights,
                lengths[j],
                biases[j],
                dilations[j],
                paddings[j],
                num_channel_indices[j],
                channel_indices[a2:b2]
            )
            X[i, a3] = temp_result[1]
            X[i, a3 + 1] = temp_result[2]
        }

        a1 = b1 + 1
        a2 = b2 + 1
        a3 = b3 + 1       
    }
}


result_x = as.data.frame(X)



In [44]:
linear_model <- import("sklearn.linear_model")


classifier = linear_model$RidgeClassifierCV(alphas=pracma::logspace(-3, 3, 10), normalize=TRUE)
classifier$fit(result_x, y_train)


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

In [45]:
arrow_head <- datasets$load_basic_motions(split="test", return_X_y=TRUE)
x_test <- arrow_head[[1]]
y_test <- arrow_head[[2]]



utils <- import("sktime.utils.validation.panel")
x_test = utils$check_X(x_test, coerce_to_numpy=TRUE)

for(i in 1 : dim(x_test)[1])
    for (o in 1 : dim(x_test)[2]) {
        x_test[i,o,] = (x_test[i,o,] - mean(x_test[i,o,])) / sd(x_test[i,o,]) + 1e-8 
    }

num_instances = dim(x_test)[1]
num_columns = dim(x_test)[2]

num_kernels = length(lengths)

X = matrix(0, num_instances, num_kernels * 2)  # 2 features per kernel

for (i in 1 : num_instances) {
    a1 = 1  # for weights
    a2 = 1  # for channel_indices
    a3 = 1  # for features

    for (j in 1 : num_kernels)
    {
        b1 = a1 + num_channel_indices[j] * lengths[j] - 1
        b2 = a2 + num_channel_indices[j] - 1
        b3 = a3 + 2 - 1
        
        if (num_channel_indices[j] == 1)
        {
            
            temp_result = apply_kernel_univariate(
                x_test[i, channel_indices[a2],],
                weights[a1:b1],
                lengths[j],
                biases[j],
                dilations[j],
                paddings[j]
            ) 
            X[i, a3] = temp_result[1]
            X[i, a3 + 1] = temp_result[2]
        }
        else
        {
            temp_weights = matrix(weights[a1 : b1], num_channel_indices[j], lengths[j])

            temp_result = apply_kernel_multivariate(
                x_test[i,,],
                temp_weights,
                lengths[j],
                biases[j],
                dilations[j],
                paddings[j],
                num_channel_indices[j],
                channel_indices[a2:b2]
            )
            X[i, a3] = temp_result[1]
            X[i, a3 + 1] = temp_result[2]
        }

        a1 = b1 + 1
        a2 = b2 + 1
        a3 = b3 + 1       
    }
}


result_x = as.data.frame(X)




In [46]:
classifier$score(result_x, y_test)

[1] 1

In [47]:
print(y_test)


 [1] "Standing"  "Standing"  "Standing"  "Standing"  "Standing"  "Standing" 
 [7] "Standing"  "Standing"  "Standing"  "Standing"  "Running"   "Running"  
[13] "Running"   "Running"   "Running"   "Running"   "Running"   "Running"  
[19] "Running"   "Running"   "Walking"   "Walking"   "Walking"   "Walking"  
[25] "Walking"   "Walking"   "Walking"   "Walking"   "Walking"   "Walking"  
[31] "Badminton" "Badminton" "Badminton" "Badminton" "Badminton" "Badminton"
[37] "Badminton" "Badminton" "Badminton" "Badminton"
